In [5]:
import requests
import folium
import numpy as np
import pandas as pd
import json

# L'URL de l'API pour donné de compteur 
limit_datacount = 100 # télécharger 100 données mais ne pas saugarder localement 
offset_datacount = 0
dateEtHeure ='2023-12-24T23'
#api avec un filtre "order by" dans l'ordre décroissant de dates dans datacount
api_url_datacount = "https://data.metropole-rouen-normandie.fr/api/explore/v2.1/catalog/datasets/eco-counter-data/records?order_by=date%20DESC&limit=" + str(limit_datacount) +"&offset=" + str(offset_datacount)

#L'URL de l'API pour localisation des sites de comptage
api_url_localisation = "https://data.metropole-rouen-normandie.fr/api/explore/v2.1/catalog/datasets/eco-counter-sites/records?limit=29"

#L'URL de l'API pour télécharger toutes les données 
api_url_export = "https://data.metropole-rouen-normandie.fr/api/explore/v2.1/catalog/datasets/eco-counter-data/exports/json?lang=fr&timezone=Europe%2FBerlin"


# #local_data telecharge depuis serveur
local_data = pd.read_json("local_data.json")

update_data = [] # initialise update_data par une liste

dateEtHeure = local_data.iloc[0]['date'][:-12] # pour avoir la forme de variable dateEtHeure identique que celle declarée dessus

#L'URL de l'API pour la mise à jour de données (where date > dateEtHeure le plus récent indiqué dans local_data, order by date descendant, timezone Europe/Berlin)
api_url_update_data ="https://data.metropole-rouen-normandie.fr/api/explore/v2.1/catalog/datasets/eco-counter-data/records?where=date%3Edate'"+dateEtHeure+"%3A00%3A00%2B01%3A00'&order_by=date%20DESC&limit="+str(limit_datacount)+"&offset="+str(offset_datacount)+"&timezone=Europe%2FBerlin"


# Effectuer une requête GET pour obtenir les données
response_updateData = requests.get(api_url_update_data) # dictionnaire (indice total_count, indice results)
response_localisation = requests.get(api_url_localisation)

if (response_updateData.status_code != 200):
   
    print(f'Échec de la requête HTTP, code de statut {response_updateData.status_code}')

if (response_localisation.status_code != 200):
   
    print(f'Échec de la requête HTTP, code de statut {response_localisation.status_code}')



# Si la requête a réussi (code de statut HTTP 200), vous pouvez accéder aux données
data_localisation = response_localisation.json()

total_count = response_updateData.json()['total_count']

print(total_count)

if (total_count != 0) : # le nombre de données mises à jour 
    while (total_count >= 100) : # data limit is 100
        update_data.extend(requests.get(api_url_update_data).json()['results'])
        total_count -= 100
        offset_datacount +=  100
        api_url_update_data ="https://data.metropole-rouen-normandie.fr/api/explore/v2.1/catalog/datasets/eco-counter-data/records?where=date%3Edate'"+dateEtHeure+"%3A00%3A00%2B01%3A00'&order_by=date%20DESC&limit="+str(limit_datacount)+"&offset="+str(offset_datacount)+"&timezone=Europe%2FBerlin"

    limit_datacount = total_count
    api_url_update_data ="https://data.metropole-rouen-normandie.fr/api/explore/v2.1/catalog/datasets/eco-counter-data/records?where=date%3Edate'"+dateEtHeure+"%3A00%3A00%2B01%3A00'&order_by=date%20DESC&limit="+str(limit_datacount)+"&offset="+str(offset_datacount)+"&timezone=Europe%2FBerlin"
    update_data.extend(requests.get(api_url_update_data).json()['results'])
    list_local_data =local_data.to_dict(orient='records')
    update_data.extend(list_local_data)
    # Utilisez json.dump() pour sauvegarder le dictionnaire dans le fichier JSON

    with open('local_data.json', 'w') as fichier_json:
        json.dump(update_data, fichier_json)


# df_update_data =pd.DataFrame(update_data)
# df_combined = pd.concat([df_update_data,local_data])


# results_datacount = local_data['results']
#results_localisation = data_localisation['results']



    



    

# print("ID:", first_result['id'])
# print("Date:", first_result['date'])
# print("Counts:", first_result['counts'])
# print("Status:", first_result['status'])
# print("Name:", first_result['name'])
# print("User Type:", first_result['user_type'])
# print("Photo URL:", first_result['photourl'])
# print("Date_j:", first_result['date_j'])


# for i in range (len(results_localisation)):
    
#     lon = results_localisation[i]['coordinates']['lon']
#     lat = results_localisation[i]['coordinates']['lat']
#     if i==0:
#         map_center = [lat, lon]
#         mymap = folium.Map(location=map_center, zoom_start=15)
#     coordinates = [lat, lon]
   
#     # Dessiner un cercle proportionnel à l'intervalle
#     radius = np.sqrt(results_localisation[i]['interval'] / np.pi)   # Ajustez le facteur selon vos besoins
#     folium.CircleMarker(location=coordinates, radius=radius, color='blue', fill=True).add_to(mymap)
#     # Afficher la carte
# mymap.save('map.html')



0
